In [3]:
import pandas as pd

test_data_df= pd.read_pickle("test_data.pkl")
test_data_df = test_data_df[["code","date"]]

len(test_data_df)

4879631

In [4]:
date_df = pd.read_excel("date_search.xlsx")
# 2️⃣ 按 'date' 列合并
merged_df = pd.merge(test_data_df, date_df, on="date", how="left")

# 3️⃣ 将 'date_str' 转换为日期格式（只保留年月日）
merged_df["date_str"] = pd.to_datetime(merged_df["date_str"]).dt.date

# 4️⃣ 查看结果
merged_df

,code,date,date_str
0,s_0,1702,2020-01-02
1,s_3079,1702,2020-01-02
2,s_4533,1702,2020-01-02
3,s_484,1702,2020-01-02
4,s_2840,1702,2020-01-02
...,...,...,...
4879626,s_2996,2803,2024-07-19
4879627,s_2995,2803,2024-07-19
4879628,s_2994,2803,2024-07-19
4879629,s_82,2803,2024-07-19


In [5]:
ticker_df = pd.read_excel("ticker_search.xlsx")

merged_df = pd.merge(merged_df, ticker_df, on="code", how="left")
merged_df

,code,date,date_str,ticker,name
0,s_0,1702,2020-01-02,000758.SZ,中色股份
1,s_3079,1702,2020-01-02,603367.SH,辰欣药业
2,s_4533,1702,2020-01-02,002573.SZ,清新环境
3,s_484,1702,2020-01-02,688009.SH,中国通号
4,s_2840,1702,2020-01-02,002759.SZ,天际股份
...,...,...,...,...,...
4879626,s_2996,2803,2024-07-19,605198.SH,安德利
4879627,s_2995,2803,2024-07-19,NaN,NaN
4879628,s_2994,2803,2024-07-19,300359.SZ,全通教育
4879629,s_82,2803,2024-07-19,NaN,NaN


In [ ]:
#merged_df.to_csv("merged_df.csv", encoding="utf-8-sig")


In [7]:
from src.xueqiu_data import getChinaStockHistoryByDate2


In [8]:
ticker_list = merged_df['ticker'].unique()
ticker_list

len(merged_df)

4879631

In [11]:
# for i,ticker in enumerate(ticker_list):
#     try:
#         print("i = ",i)
#         price_df = getChinaStockHistoryByDate2(ticker, start_date = '2020-01-02')
#         price_df['date_str'] = pd.to_datetime(price_df['date_str']).dt.date
#         price_df.to_csv("data/"+ ticker +".csv",index=None)
#     except:
#         print("Error: ",ticker)
#         pass

In [23]:
# 假设已有: merged_df, ticker_list
# 1) 统一日期类型
merged_df['date_str'] = pd.to_datetime(merged_df['date_str']).dt.date

# 2) 先整合所有 price_df
price_frames = []
missing_files = []  # 记录缺失或读取失败的ticker，便于排查

for i, ticker in enumerate(ticker_list):
    print("i = ", i, " data/" + str(ticker) + ".csv")
    path = f"data/{ticker}.csv"
    try:
        df = pd.read_csv(path)
        # 只保留需要的列，并补上 ticker 列
        if 'date_str' not in df.columns or 'y_pred' not in df.columns:
            raise ValueError(f"{path} 缺少必要列: 'date_str' 或 'y_pred'")
        df = df[['date_str', 'y_pred']].copy()
        df['date_str'] = pd.to_datetime(df['date_str']).dt.date
        df['ticker'] = ticker
        price_frames.append(df)
    except Exception as e:
        missing_files.append((ticker, str(e)))
        # 不中断，继续下一个ticker
        continue

# 若一个都没读到，直接给出空结果以避免后续报错
if len(price_frames) == 0:
    print("未读取到任何 price_df，请检查数据文件。")
    output = merged_df.copy()
    output = output.sort_values(['ticker','date_str'], na_position='last').reset_index(drop=True)
    output['id'] = range(len(output))
    output = output[['id','date_str','y_pred']] if 'y_pred' in output.columns else output[['id','date_str']]
    output.to_csv("output.csv", encoding="utf-8-sig", index=False)
else:
    # 3) 拼接为一个大的 all_price_df，并去重（如有）
    all_price_df = pd.concat(price_frames, ignore_index=True)
    # 去重策略：同一 (ticker,date_str) 保留最后一条，或改成 first/mean 按需
    all_price_df = (all_price_df
                    .sort_values(['ticker','date_str'])
                    .drop_duplicates(subset=['ticker','date_str'], keep='last'))

    # 4) 与 merged_df 一次性合并（左连接，保留 merged_df 所有行）
    merged_once = pd.merge(
        merged_df,
        all_price_df[['ticker','date_str','y_pred']],
        how='left',
        on=['ticker','date_str']
    )

    # 5) 排序、生成 id、导出（保持你原来的导出字段顺序与命名）
    output = merged_once.sort_values(['code','date']).reset_index(drop=True)
    print(len(output))
    output['id'] = range(len(output))
    output = output[['id','code','date','y_pred']]
    output['y_pred'] = output['y_pred'].fillna(0)
    output.to_csv("output.csv", encoding="utf-8-sig", index=False)

# 可选：打印读取失败的文件，便于排查
if missing_files:
    print("以下ticker文件读取失败：")
    for tkr, err in missing_files:
        print(f"- {tkr}: {err}")


i =  0  data/000758.SZ.csv
i =  1  data/603367.SH.csv
i =  2  data/002573.SZ.csv
i =  3  data/688009.SH.csv
i =  4  data/002759.SZ.csv
i =  5  data/688002.SH.csv
i =  6  data/603729.SH.csv
i =  7  data/603815.SH.csv
i =  8  data/000070.SZ.csv
i =  9  data/002053.SZ.csv
i =  10  data/300126.SZ.csv
i =  11  data/002885.SZ.csv
i =  12  data/600330.SH.csv
i =  13  data/300523.SZ.csv
i =  14  data/603678.SH.csv
i =  15  data/300252.SZ.csv
i =  16  data/300496.SZ.csv
i =  17  data/603233.SH.csv
i =  18  data/000858.SZ.csv
i =  19  data/nan.csv
i =  20  data/300501.SZ.csv
i =  21  data/600167.SH.csv
i =  22  data/300293.SZ.csv
i =  23  data/002889.SZ.csv
i =  24  data/002092.SZ.csv
i =  25  data/300792.SZ.csv
i =  26  data/600185.SH.csv
i =  27  data/601566.SH.csv
i =  28  data/300294.SZ.csv
i =  29  data/300077.SZ.csv
i =  30  data/601958.SH.csv
i =  31  data/600072.SH.csv
i =  32  data/300785.SZ.csv
i =  33  data/000803.SZ.csv
i =  34  data/603099.SH.csv
i =  35  data/300484.SZ.csv
i =  36 

In [24]:
output

,id,code,date,y_pred
0,0,s_0,1702,0.134716
1,1,s_0,1703,-0.503086
2,2,s_0,1704,-0.427160
3,3,s_0,1705,-0.098491
4,4,s_0,1706,0.066907
...,...,...,...,...
4879626,4879626,s_999,2799,-0.591880
4879627,4879627,s_999,2800,-0.814359
4879628,4879628,s_999,2801,-0.221359
4879629,4879629,s_999,2802,-0.250942
